## Key Word: California in ENRON Emails to CSV 

Note: some functions are similar across the files since most of these scripts were run concurrently

In [1]:
import pandas as pd
import os
from collections import Counter

In [3]:
kmetadata = pd.DataFrame()
kexchange = pd.DataFrame()

In [4]:
# starting point
contents = 'C:/Users/Aneesha2/Desktop/enron_contents/maildir'

# pulling employee folders
for emp_folder in os.listdir(contents): #sample
    # sent emails
    emails_path = f'C:/Users/Aneesha2/Desktop/enron_contents/maildir/{emp_folder}/sent/'
    try:
        for email in os.listdir(emails_path):
            
            # lists out every email
            email_path = emails_path + f'{email}'

            # open file
            mail = open(email_path,"r")
            # each line is a line of the text file
            lines = mail.readlines()
            mail.close()

            # 1 element in each of these arrays
            message_id = [line for line in lines if 'Message-ID:' in line]    
            date = [line for line in lines if 'Date:' in line]   
            from_email = [line for line in lines if 'From:' in line]    
            subject = [line for line in lines if 'Subject:' in line]
            encoding = [line for line in lines if 'Content-Transfer-Encoding:' in line]

            # Getting the body of the email
            body_start = [i for i,line in enumerate(lines) if 'X-FileName:' in line][0]
            body = lines[body_start + 2: ]
            body = ' '.join(body)
            
            # checks if califorina is mentioned in the email
            if 'alifornia' in body:

                # list of words
                body = body.split()

                # count number of words
                body_len = len(body)


                to_email = [line for line in lines if 'To:' in line]   
                emailed = []
                for word in body:
                    if '@' in str(word):
                        emailed.append(word.replace(' ', ''))

                if len(to_email) < len(emailed):
                    to_email = emailed


                kexchange = kexchange.append({'message_id' : message_id,
                                            'from_email' : from_email,
                                           'to_email' : to_email},
                                          ignore_index = True)

                kmetadata = kmetadata.append({'message_id' : message_id,
                                            'date' : date,
                                            'length' : body_len,
                                            'subject' : subject,
                                            'encoding' : encoding}, 
                                           ignore_index = True)
        
    except:
        print(emp_folder)
    
    print(emails_path)

C:/Users/Aneesha2/Desktop/enron_contents/maildir/allen-p/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/arnold-j/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/arora-h/sent/
badeer-r
C:/Users/Aneesha2/Desktop/enron_contents/maildir/badeer-r/sent/
bailey-s
C:/Users/Aneesha2/Desktop/enron_contents/maildir/bailey-s/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/bass-e/sent/
baughman-d
C:/Users/Aneesha2/Desktop/enron_contents/maildir/baughman-d/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/beck-s/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/benson-r/sent/
blair-l
C:/Users/Aneesha2/Desktop/enron_contents/maildir/blair-l/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/brawner-s/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/buy-r/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/campbell-l/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/carson-m/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/cash-m/sent/
causholli-

C:/Users/Aneesha2/Desktop/enron_contents/maildir/stclair-c/sent/
steffes-j
C:/Users/Aneesha2/Desktop/enron_contents/maildir/steffes-j/sent/
stepenovitch-j
C:/Users/Aneesha2/Desktop/enron_contents/maildir/stepenovitch-j/sent/
stokley-c
C:/Users/Aneesha2/Desktop/enron_contents/maildir/stokley-c/sent/
storey-g
C:/Users/Aneesha2/Desktop/enron_contents/maildir/storey-g/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/sturm-f/sent/
swerzbin-m
C:/Users/Aneesha2/Desktop/enron_contents/maildir/swerzbin-m/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/symes-k/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/taylor-m/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/tholt-j/sent/
thomas-p
C:/Users/Aneesha2/Desktop/enron_contents/maildir/thomas-p/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/townsend-j/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/tycholiz-b/sent/
C:/Users/Aneesha2/Desktop/enron_contents/maildir/ward-k/sent/
watson-k
C:/Users/Aneesha2/Des

In [8]:
kexchange['from_email'] = kexchange['from_email'].apply(lambda data: data[0].replace('From:','').replace('\n', ''))
kexchange['message_id'] = kexchange['message_id'].apply(lambda data: data[0].replace('Message-ID:','').replace('\n', ''))
kexchange['new_to'] = kexchange['to_email'].apply(lambda ex : ex[0].replace('To:', '').replace('\n', '').replace(' ', '').replace('>','').replace('<','').split(','))

In [11]:
# Reformatting the dataset
new_exchange = pd.DataFrame()

for index, row in kexchange.iterrows():
    for item in row['new_to']:
        if '@' in item:
            new_exchange = new_exchange.append({'from_email' : row.from_email,
                                                'message_id' : row.message_id,
                                                'to_email' : item}, 
                                               ignore_index = True)

In [15]:
kmetadata['date'] = kmetadata['date'].apply(lambda data: data[0].replace('Date:','').replace('\n', ''))
kmetadata['encoding'] = kmetadata['encoding'].apply(lambda data: data[0].replace('Content-Transfer-Encoding:','').replace('\n', ''))
kmetadata['message_id'] = kmetadata['message_id'].apply(lambda data: data[0].replace('Message-ID:','').replace('\n', ''))
kmetadata['subject'] = kmetadata['subject'].apply(lambda data: data[0].replace('Subject:','').replace('\n', ''))

In [17]:
ca = pd.merge(new_exchange, kmetadata, on='message_id', how='left')

In [18]:
ca.head()

,from_email,message_id,to_email,date,encoding,length,subject
0,phillip.allen@enron.com,<7937898.1075855682041.JavaMail.evans@thyme>,Allen/HOU/ECT@ECT,"Wed, 6 Sep 2000 04:46:00 -0700 (PDT)",7bit,1173.0,Wow
1,phillip.allen@enron.com,<21697489.1075855682172.JavaMail.evans@thyme>,rich@pira.com,"Thu, 31 Aug 2000 07:36:00 -0700 (PDT)",7bit,28.0,Re: Western Gas Market Report -- Draft
2,phillip.allen@enron.com,<27222590.1075855684071.JavaMail.evans@thyme>,matthew.lenhart@enron.com,"Mon, 12 Jun 2000 03:56:00 -0700 (PDT)",7bit,310.0,Thoughts on Presentation
3,phillip.allen@enron.com,<19261871.1075855684502.JavaMail.evans@thyme>,kenneth.shulklapper@enron.com,"Fri, 28 Apr 2000 02:05:00 -0700 (PDT)",7bit,210.0,Re: SW Gas
4,phillip.allen@enron.com,<30795938.1075855685528.JavaMail.evans@thyme>,Miller/Corp/Enron@ENRON,"Mon, 21 Feb 2000 00:07:00 -0800 (PST)",quoted-printable,402.0,PIRA's California/Southwest Gas Pipeline Study


In [19]:
ca.to_csv('ca.csv')